## working with drone

In [34]:
import cv2
import numpy as np
import os 
import time
import socket

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);
font = cv2.FONT_HERSHEY_SIMPLEX
#iniciate id counter
id = 0
# names related to ids: example ==> Marcelo: id=1,  etc
names = ["None", "front", "top_view", "right", "left"] 
# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video widht
cam.set(4, 480) # set video height
# Define min window size to be recognized as a face
minW = 0.01*cam.get(3)
minH = 0.01*cam.get(4)
# ###
# tello_ip = '192.168.10.1'
# tello_port = 8889
# tello_address = (tello_ip, tello_port)
# mypc_address = ('', 9001)
# socket = socket.socket (socket.AF_INET, socket.SOCK_DGRAM)
# socket.bind (mypc_address)
# socket.sendto ('command'.encode (' utf-8 '), tello_address)
# socket.sendto ('streamon'.encode (' utf-8 '), tello_address)
# print ("Start streaming")
# capture = cv2.VideoCapture ('udp://0.0.0.0:11111',cv2.CAP_FFMPEG)
# if not capture.isOpened():
#     capture.open('udp:/0.0.0.0:11111')
# ###

def dead_band(x, y, w, h, old_x, old_y, old_w, old_h, epsilon):
    if (abs(old_x - x) < epsilon):
        x = old_x;
    if (abs(old_y - y) < epsilon):
        y = old_y;
    if (abs(old_w - w) < epsilon):
        w = old_w;
    if (abs(old_h - h) < epsilon):
        h = old_h;
    
    return x, y, w, h

def find_directions(x, y, center_x, center_y):
    if ( x < center_x) and (y < center_y):
        return "down", "right"
    elif ( x < center_x) and (y > center_y):
        return "up", "right"
    elif ( x > center_x) and (y < center_y):
        return "down", "left"
    elif ( x > center_x) and (y > center_y):
        return "up", "left"
    else:
        return "NA", "NA"
    

old_x, old_y, old_w, old_h = 0, 0, 0, 0

while True:
    ret, img =cam.read()
#     ret, img =capture.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,scaleFactor = 1.2,minNeighbors = 5,minSize = (int(minW), int(minH)),)
    
    for(x,y,w,h) in faces:
        x, y, w, h = dead_band(x, y, w, h, old_x, old_y, old_w, old_h, 8);
        
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        if not (confidence < 80):
            continue
        cv2.rectangle(img, (x,y), (x+w,y+h), (255, 0, 0), 2)
        # If confidence is less them 100 ==> "0" : perfect match 
        if (confidence < 100):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        cx, cy = x+w//2, y+h//2
        
        cv2.line(img, (cx, cy), (img.shape[1]//2, img.shape[0]//2), (0, 255, 0) , 5) 
#         cv2.rectangle(img, (x,y), (x+w,y+h), (255, 0, 0), 2)
        cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
        cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
        
        res_x, res_y = find_directions(cx, cy, img.shape[1]//2, img.shape[0]//2)
        
        cv2.putText(img, "move {} and {} ".format(res_x, res_y), (30,460), font, 1, (0,0,0), 1)  
        cv2.putText(img, "w={}, h={}".format(str(h), str(w)), (30,30), font, 1, (0,0,0), 1)  
        old_x, old_y, old_w, old_h = x, y, w, h
    
    cv2.imshow('camera',img) 
    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()
# capture.release ()
# cv2.destroyAllWindows ()
# socket.sendto ('streamoff'.encode (' utf-8 '), tello_address)



 [INFO] Exiting Program and cleanup stuff


In [ ]:
import cv2
import numpy as np
import os 
import time
import socket

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);
font = cv2.FONT_HERSHEY_SIMPLEX
#iniciate id counter
id = 0
# names related to ids: example ==> Marcelo: id=1,  etc
names = ["None", "front", "top_view"] 
# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video widht
cam.set(4, 480) # set video height
# Define min window size to be recognized as a face
minW = 0.01*cam.get(3)
minH = 0.01*cam.get(4)
###
tello_ip = '192.168.10.1'
tello_port = 8889
tello_address = (tello_ip, tello_port)
mypc_address = ('', 9001)
socket = socket.socket (socket.AF_INET, socket.SOCK_DGRAM)
socket.bind (mypc_address)
socket.sendto ('command'.encode (' utf-8 '), tello_address)
socket.sendto ('streamon'.encode (' utf-8 '), tello_address)
print ("Start streaming")
capture = cv2.VideoCapture ('udp://0.0.0.0:11111',cv2.CAP_FFMPEG)
if not capture.isOpened():
    capture.open('udp:/0.0.0.0:11111')
###

while True:
#     ret, img =cam.read()
    ret, img =capture.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,scaleFactor = 1.2,minNeighbors = 5,minSize = (int(minW), int(minH)),)
    
    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255, 0, 0), 2)
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
#         if id != 2 or not (confidence < 80):
#             continue
        # If confidence is less them 100 ==> "0" : perfect match 
        if (confidence < 100):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        cx, cy = x+w//2, y+h//2
        cv2.line(img, (cx, cy), (img.shape[1]//2, img.shape[0]//2), (0, 255, 0) , 5) 
        cv2.rectangle(img, (x,y), (x+w,y+h), (255, 0, 0), 2)
        cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
        cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
        cv2.putText(img, "{}, {}".format(str(h), str(w)), (30,30), font, 1, (255,255,0), 1)  
    
    cv2.imshow('camera',img) 
    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
# cam.release()
# cv2.destroyAllWindows()
capture.release ()
cv2.destroyAllWindows ()
socket.sendto ('streamoff'.encode (' utf-8 '), tello_address)
